labelme) Only manual labeling on static images
We can manually draw the target area (AOI) on the image and then generate a JSON file to record the locations of these areas.
Only extract video frames frame by frame and manually label the AOI on each frame.

try to analyze the gaze data with in AOI Plan A

Step 1: Automatic tracking + visualise the display position (dynamic ROI)
You manually click on the 4 corners in the first frame → initialise the display position
Use optical flow to track the corners of the display in each frame
Save the ROI (coordinates of the four corners) of the display in each frame
Visualise every few frames to help you manually check the tracking results
Finally, save the ROI of all frames in a CSV or JSON file for subsequent use

In [ ]:
pip install opencv-python-headless numpy pandas shapely matplotlib

read gaze data

In [2]:
import pandas as pd

# Load the uploaded gaze CSV file
gaze_file_path = "/Users/jiaonahu/Documents/eye-tracking/sub_RJ70HQ/exports/000/gaze_positions.csv"
gaze_data = pd.read_csv(gaze_file_path)

# Display the first few rows to understand the structure
gaze_data.head()

,gaze_timestamp,world_index,confidence,norm_pos_x,norm_pos_y,base_data,gaze_point_3d_x,gaze_point_3d_y,gaze_point_3d_z,eye_center0_3d_x,...,eye_center0_3d_z,gaze_normal0_x,gaze_normal0_y,gaze_normal0_z,eye_center1_3d_x,eye_center1_3d_y,eye_center1_3d_z,gaze_normal1_x,gaze_normal1_y,gaze_normal1_z
0,27552.401964,1,0.810659,0.631600,0.015143,27552.368713-0 27552.435214-1,12.220782,33.768872,60.316149,31.925014,...,0.387049,-0.271092,0.434771,0.858768,-37.624465,-9.240443,-2.938625,0.541557,0.508180,0.669679
1,27552.403943,1,0.337533,3.007644,-1.288352,27552.403943-1,397.701637,212.718724,103.010951,NaN,...,NaN,NaN,NaN,NaN,-37.624465,-9.240443,-2.938625,0.870652,0.443918,0.211899
2,27552.404224,1,0.444916,0.460274,0.185104,27552.404224-0,-28.548249,169.721647,466.767724,31.925014,...,0.387049,-0.120947,0.339600,0.932761,NaN,NaN,NaN,NaN,NaN,NaN
3,27552.435201,2,0.846872,0.655470,0.082286,27552.435187-0 27552.435214-1,15.872700,31.984810,66.312535,31.925014,...,0.387049,-0.203939,0.341408,0.917524,-37.624465,-9.240443,-2.938625,0.541557,0.508180,0.669679
4,27552.451869,2,0.867987,0.669580,0.109007,27552.468523-0 27552.435214-1,18.201822,31.475342,69.715881,31.925014,...,0.387049,-0.164670,0.300020,0.939613,-37.624465,-9.240443,-2.938625,0.541557,0.508180,0.669679


Import libraries and setup Show first frame and select 4 corners manually

In [12]:
import cv2
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

# === Modify the path to your video ===
VIDEO_PATH = '/Users/jiaonahu/Documents/eye-tracking/sub_RJ70HQ/exports/000/gaze_overlay_video.mp4'  # Replace with your actual path
OUTPUT_VIS_FOLDER = '/Users/jiaonahu/Desktop/monitor_tracking_visuals'
OUTPUT_TRACKING_CSV = '/Users/jiaonahu/Desktop/monitor_tracking.csv'

# Video settings
VIDEO_WIDTH = 640
VIDEO_HEIGHT = 480
FPS = 30
VISUALIZE_EVERY_N = 30  # Save a frame with visualization every N frames

# For storing clicked points
clicked_points = []

# Mouse callback to select 4 corners
def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN and len(clicked_points) < 4:
        clicked_points.append([x, y])
        print(f"Point {len(clicked_points)}: ({x}, {y})")

# Open video and read the first frame
cap = cv2.VideoCapture(VIDEO_PATH)
ret, first_frame = cap.read()
if not ret:
    raise ValueError("Could not read video.")

first_frame = cv2.resize(first_frame, (VIDEO_WIDTH, VIDEO_HEIGHT))

# Create OpenCV window and set mouse callback
cv2.namedWindow("Select Monitor Corners")
cv2.setMouseCallback("Select Monitor Corners", click_event)

print("👉 Please click the 4 corners of the monitor (clockwise or counterclockwise).")
print("✅ After clicking all 4, click on the image window and press any key to continue.")

# Main loop for selecting points
while True:
    temp_frame = first_frame.copy()
    for pt in clicked_points:
        cv2.circle(temp_frame, tuple(pt), 5, (0, 255, 0), -1)
    cv2.imshow("Select Monitor Corners", temp_frame)

    if len(clicked_points) == 4:
        key = cv2.waitKey(0)  # Wait for any key press
        if key != -1:
            break  # Exit loop after key press
    elif cv2.waitKey(1) & 0xFF == 27:  # ESC to quit early
        print("❌ Selection cancelled.")
        break

cv2.destroyAllWindows()

# Safety check
if len(clicked_points) != 4:
    raise ValueError("You must click exactly 4 points.")

print("✅ Monitor corners selected successfully:")
for i, pt in enumerate(clicked_points, 1):
    print(f"  Corner {i}: {pt}")

👉 Please click the 4 corners of the monitor (clockwise or counterclockwise).
✅ After clicking all 4, click on the image window and press any key to continue.
Point 1: (527, 192)
Point 2: (79, 190)
Point 3: (76, 442)
Point 4: (523, 463)
✅ Monitor corners selected successfully:
  Corner 1: [527, 192]
  Corner 2: [79, 190]
  Corner 3: [76, 442]
  Corner 4: [523, 463]


Track monitor corners using optical flow + save visualization

In [13]:
os.makedirs(OUTPUT_VIS_FOLDER, exist_ok=True)

prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
prev_pts = np.array(clicked_points, dtype=np.float32).reshape(-1, 1, 2)

lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

tracked_corners = []
frame_idx = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (VIDEO_WIDTH, VIDEO_HEIGHT))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    pts, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, **lk_params)
    tracked_corners.append([frame_idx] + pts.reshape(-1).tolist())

    # Visualization
    if frame_idx % VISUALIZE_EVERY_N == 0:
        vis_frame = frame.copy()
        pts_int = pts.reshape(-1, 2).astype(int)
        for i in range(4):
            cv2.circle(vis_frame, tuple(pts_int[i]), 5, (0, 255, 0), -1)
            cv2.line(vis_frame, tuple(pts_int[i]), tuple(pts_int[(i + 1) % 4]), (255, 0, 0), 2)
        vis_path = os.path.join(OUTPUT_VIS_FOLDER, f"frame_{frame_idx:05d}.png")
        plt.imsave(vis_path, vis_frame[:, :, ::-1])

    # Update
    prev_gray = gray.copy()
    prev_pts = pts
    frame_idx += 1

cap.release()

columns = ['frame',
           'x1', 'y1',
           'x2', 'y2',
           'x3', 'y3',
           'x4', 'y4']
tracked_df = pd.DataFrame(tracked_corners, columns=columns)
tracked_df.to_csv(OUTPUT_TRACKING_CSV, index=False)

print(f"Tracking data saved to {OUTPUT_TRACKING_CSV}")
print(f"Visual frames saved to folder: {OUTPUT_VIS_FOLDER}")


Tracking data saved to /Users/jiaonahu/Desktop/monitor_tracking.csv
Visual frames saved to folder: /Users/jiaonahu/Desktop/monitor_tracking_visuals


Plan C Manually label some keyframes + interpolation
Plan D Use object detection models (e.g., YOLO) to automatically detect the display

## now try with plan D train the Yolo

In [1]:
pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.3/981.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 11.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 11.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 11.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.3
    Uninstalling numpy-2.2.3:
      Successfully uninstalled numpy-2.2.3
Note: you may need to restart the kernel to use updated packages.


In [ ]:

from ultralytics import YOLO
model = YOLO("yolov8n.pt")  # load a pretrained YOLOv8 nano model

In [1]:
import numpy
print("✅ NumPy version:", numpy.__version__)

✅ NumPy version: 1.26.4


In [2]:
import cv2
import os

# === Path to your video ===
VIDEO_PATH = '/Users/jiaonahu/Documents/eye-tracking/sub_RJ70HQ/exports/000/gaze_overlay_video.mp4'

# === Output folder to save extracted frames ===
OUTPUT_FOLDER = '/Users/jiaonahu/Desktop/yolo_monitor_frames'  # Change this to any writable path on your system

# === Frame extraction settings ===
FRAME_INTERVAL_SECONDS = 3  # Extract one frame every 3 seconds

# === Create the output folder if it doesn't exist ===
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# === Open the video file ===
cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)  # Get frames per second of the video
interval_frames = int(fps * FRAME_INTERVAL_SECONDS)

frame_idx = 0
saved_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Save frame every N frames
    if frame_idx % interval_frames == 0:
        filename = f"frame_{frame_idx:05d}.jpg"
        path = os.path.join(OUTPUT_FOLDER, filename)
        cv2.imwrite(path, frame)
        saved_count += 1

    frame_idx += 1

cap.release()
print(f"✅ Extracted {saved_count} frames to: {OUTPUT_FOLDER}")


✅ Extracted 344 frames to: /Users/jiaonahu/Desktop/yolo_monitor_frames


In [7]:
import json
import os
import pandas as pd

# Set this to the folder where your .json files are located
json_folder = "/Users/jiaonahu/Desktop/yolo_monitor_frames"

# Output CSV path
output_csv_path = "/Users/jiaonahu/Desktop/monitor_corners.csv"

# Collect all JSON files
json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

# Prepare list to store data
data = []

for json_file in json_files:
    json_path = os.path.join(json_folder, json_file)
    with open(json_path, 'r') as f:
        annotation = json.load(f)

    image_name = annotation.get("imagePath", json_file.replace(".json", ".jpg"))
    shapes = annotation.get("shapes", [])

    for shape in shapes:
        if shape["label"].lower() == "monitor" and len(shape["points"]) == 4:
            # Flatten the four points (x1, y1, x2, y2, ...)
            flat_points = [coord for point in shape["points"] for coord in point]
            row = [image_name] + flat_points
            data.append(row)
            break  # only use the first "monitor" polygon per image

# Create DataFrame
columns = ["image", "x1", "y1", "x2", "y2", "x3", "y3", "x4", "y4"]
df = pd.DataFrame(data, columns=columns)

# Save to CSV
df.to_csv(output_csv_path, index=False)
print(f"✅ Saved corner data to: {output_csv_path}")
print(f"🖼️ Total annotated images: {len(df)}")



✅ Saved corner data to: /Users/jiaonahu/Desktop/monitor_corners.csv
🖼️ Total annotated images: 46


In [10]:
# Monitor Corner Regression Training Script
# Requirements: torch, torchvision, pandas, opencv-python

import os
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# === Configuration ===
CSV_PATH = "/Users/jiaonahu/Desktop/monitor_corners.csv"
IMG_FOLDER = "/Users/jiaonahu/Desktop/yolo_monitor_frames"
NUM_EPOCHS = 50
BATCH_SIZE = 8
LEARNING_RATE = 1e-4
IMG_SIZE = 224
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Dataset ===
class MonitorCornerDataset(Dataset):
    def __init__(self, csv_path, img_folder, transform=None):
        self.data = pd.read_csv(csv_path)
        self.img_folder = img_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.img_folder, row['image'])
        image = Image.open(img_path).convert("RGB")

        keypoints = row[1:].values.astype('float32')

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(keypoints)

# === Data Augmentation / Transform ===
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

# === Dataset and DataLoader ===
dataset = MonitorCornerDataset(CSV_PATH, IMG_FOLDER, transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# === Model ===
class CornerRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.resnet18(pretrained=True)
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, 8)

    def forward(self, x):
        return self.backbone(x)

model = CornerRegressionModel().to(DEVICE)
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# === Training Loop ===
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    for images, targets in dataloader:
        images = images.to(DEVICE)
        targets = targets.to(DEVICE)

        preds = model(images)
        loss = criterion(preds, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {avg_loss:.4f}")

# === Save Model ===
MODEL_PATH = "monitor_corner_model.pt"
torch.save(model.state_dict(), MODEL_PATH)
print(f"\n✅ Model saved to: {MODEL_PATH}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/jiaonahu/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>